In [ ]:
import os
from dotenv import load_dotenv
from ragas import EvaluationDataset
from ragas.llms import LangchainLLMWrapper
from langchain_neo4j import Neo4jGraph
from ragas.messages import (
    HumanMessage,
    ToolCall
)
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_ollama import ChatOllama
from src.grag import (
    create_hybrid_cypher_retriever_tool,
    create_text2cypher_retriever_tool,
    graph_rag_eval_workflow,
    FallbackToolCalling
)


load_dotenv(".env")

True

In [ ]:
URI = os.environ["DATABASE_HOST"]
DATABASE = os.environ["DATABASE_SMALL"]
USERNAME = os.environ["DATABASE_USERNAME"]
PASSWORD = os.environ["DATABASE_PASSWORD"]
DATABASE = os.environ["DATABASE_SMALL"]

neo4j_config = {
    "DATABASE_NAME": DATABASE,
    "ARTICLE_VECTOR_INDEX_NAME": os.environ["ARTICLE_VECTOR_INDEX_NAME"],
    "ARTICLE_FULLTEXT_INDEX_NAME": os.environ["ARTICLE_FULLTEXT_INDEX_NAME"],
    "DEFINITION_VECTOR_INDEX_NAME": os.environ["DEFINITION_VECTOR_INDEX_NAME"],
    "DEFINITION_FULLTEXT_INDEX_NAME": os.environ["DEFINITION_FULLTEXT_INDEX_NAME"],
}

# Ngga jadi pakai ini karena Text2Cypher mintannya harus pakai Neo4jGraph
# neo4j_driver = GraphDatabase.driver(uri=URI, auth=(USERNAME, PASSWORD))

neo4j_graph = Neo4jGraph(
    url=URI,
    username=USERNAME,
    password=PASSWORD,
    database=DATABASE,
    enhanced_schema=True
)

neo4j_driver = neo4j_graph._driver  # Ambil driver nya kaya gini

embedder_model = HuggingFaceEmbeddings(model_name=os.environ["EMBEDDING_MODEL"])

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.0,
    api_key=os.environ["GOOGLE_API_KEY"]
)

gemini_llm_evaluator = LangchainLLMWrapper(llm)

# llm = ChatOllama(model="llama3.1:8b-instruct-q4_0", temperature=0.0)
# llm = ChatOllama(model="qwen2.5:7b-instruct-q4_K_M", temperature=0.0)
# llm = ChatOllama(model="cogito:8b-v1-preview-llama-q4_K_M", temperature=0.0)  # Generate jawaban akhir nya lama banget
# llm = ChatOllama(model="granite3.3:2b", temperature=0.0)  # Bisa tapi hasilnya aneh, coba yang model 8B nya dulu

In [ ]:
total_definition_limit = 5

hybrid_cypher_retriever = create_hybrid_cypher_retriever_tool(
    embedder_model=embedder_model,
    neo4j_driver=neo4j_graph._driver,  # Coba sederhakan jadi neo4j_graph saja
    neo4j_config=neo4j_config,
    total_definition_limit=total_definition_limit,
    top_k_initial_article=5,
    max_k_expanded_article=-1,
    total_article_limit=None,
    ranker="linear",
    alpha=0.5
)

text2cypher_retriever = create_text2cypher_retriever_tool(
    neo4j_graph=neo4j_graph,
    embedder_model=embedder_model,
    cypher_llm=llm,
    qa_llm=llm,
    skip_qa_llm=True,
    verbose=False
)

In [ ]:
dataset = [
    {
        "user_input": (
            "Dalam pembangunan infrastruktur telekomunikasi, bagaimana cara perhitungan persentase TKDN untuk belanja modal atau capital expenditure (Capex) bisa dilakukan? Carikan caranya"
        ),
        "reference_tool_calls": (
            "hybrid_cypher_retriever"
        ),
        "reference": """
            Persentase TKDN untuk belanja modal (capex) dihitung berdasarkan perbandingan antara total belanja modal dikurangi belanja modal komponen luar negeri terhadap total belanja modal, atau perbandingan antara total belanja modal komponen dalam negeri terhadap total belanja modal. Formula yang digunakan adalah: % TKDN (Capex) = ((Total Capex - Capex Komponen Luar Negeri) x 100%) / Total Capex atau % TKDN (Capex) = (Capex Komponen Dalam Negeri x 100%) / Total Capex. Belanja modal ini meliputi biaya untuk engineering, material/perangkat telekomunikasi, tenaga kerja, alat kerja, dan jasa pendukung pada pembangunan infrastruktur telekomunikasi.
            """.strip(),
        "reference_contexts": [
            "Peraturan Menteri Komunikasi dan Informatika Nomor 41/PER/M.KOMINFO/10/2009 Tahun 2009 tentang Tata Cara Penilaian Pencapaian Tingkat Komponen Dalam Negeri pada Penyelenggaraan Telekomunikasi, BAB III - TINGKAT KOMPONEN DALAM NEGERI, Pasal 3:\n(1) Persentase TKDN untuk belanja modal (capex) sebagaimana dimaksud dalam Pasal 2 dihitung berdasarkan :\n\ta. perbandingan antara total belanja modal (capex) dikurangi belanja modal (capex) Komponen luar negeri terhadap total belanja modal (capex); atau\n\tb. perbandingan antara total belanja modal (capex) Komponen dalam negeri terhadap total belanja modal (capex);\n(2) Formula TKDN belanja modal (capex) sebagaimana dimaksud pada ayat (1) didasarkan pada rasio sebagai berikut :\n$$\n\\% TKDN (Capex) = \\frac{(Total\\ Capex - Capex\\ Komponen\\ Luar\\ Negeri) \\times 100\\%}{Total\\ Capex}\n$$\natau\n$$\n\\% TKDN (Capex) = \\frac{Capex\\ Komponen\\ Dalam\\ Negeri \\times 100\\%}{Total\\ Capex}\n$$\n(3) Belanja modal (capex) sebagaimana dimaksud pada ayat (1) meliputi biaya pengeluaran untuk :\n\ta. engineering untuk kegiatan perekayasaan pembangunan infrastruktur telekomunikasi;\n\tb. material/perangkat telekomunikasi yang dipergunakan pada pembangunan infrastruktur telekomunikasi;\n\tc. tenaga kerja pada pembangunan infrastruktur telekomunikasi;\n\td. alat kerja yang dipergunakan pada pembangunan infrastruktur telekomunikasi;\n\te. biaya jasa pendukung pada pembangunan infrastruktur telekomunikasi.\n(4) Format dan formula perhitungan TKDN sebagaimana dimaksud pada ayat (2) dan ayat (3) adalah sebagai berikut :\n| No. | Komponen Capex                        | Nilai Capex (Rp) | Capex KDN (Rp) | Capex KLN (Rp) | % TKDN                          |\n|-----|----------------------------------------|------------------|----------------|----------------|--------------------------------|\n|     | (a)                                    | (b)              | (c)            | (d)            | (e) = (c)/(b) atau (b-d)/(b)  |\n| 1.  | Engineering                            | b1               | c1             | d1             | e1 = c1/b1 atau (b1-d1)/b1    |\n| 2.  | Material/Perangkat Telekomunikasi      | b2               | c2             | d2             | e2 = c2/b2 atau (b2-d2)/b2    |\n| 3.  | Tenaga Kerja                           | b3               | c3             | d3             | e3 = c3/b3 atau (b3-d3)/b3    |\n| 4.  | Alat Kerja                             | b4               | c4             | d4             | e4 = c4/b4 atau (b4-d4)/b4    |\n| 5.  | Jasa Pendukung                         | b5               | c5             | d5             | e5 = c5/b5 atau (b5-d5)/b5    |\n|     | **Total**                              | **Σb = b1+...+b5** | **Σc = c1+...+c5** | **Σd = d1+...+d5** | **Σe = Σc/Σb atau (Σb-Σd)/Σb** |",
            "Peraturan Menteri Komunikasi dan Informatika Nomor 41/PER/M.KOMINFO/10/2009 Tahun 2009 tentang Tata Cara Penilaian Pencapaian Tingkat Komponen Dalam Negeri pada Penyelenggaraan Telekomunikasi, BAB III - TINGKAT KOMPONEN DALAM NEGERI, Pasal 7:\n(1) Besarnya nilai belanja modal (capex) Komponen dalam negeri yang dihitung dalam rupiah dihitung berdasarkan formula sebagai berikut :\n\tNilai Capex KDN (Rp.) = (nilai perolehan komponen Capex(Rp.)) × % TKDN komponen Capex\n(2) Dalam hal penyelenggara telekomunikasi melakukan pembelanjaan material/perangkat telekomunikasi dengan menggunakan mata uang selain rupiah, maka mata uang dimaksud harus dikonversikan dalam hitungan rupiah berdasarkan rate Bank Indonesia (BI) yang berlaku pada saat pembelanjaan dilakukan.",
            "Peraturan Menteri Komunikasi dan Informatika Nomor 41/PER/M.KOMINFO/10/2009 Tahun 2009 tentang Tata Cara Penilaian Pencapaian Tingkat Komponen Dalam Negeri pada Penyelenggaraan Telekomunikasi, BAB IV - PENILAIAN SENDIRI DAN VERIFIKASI TINGKAT KOMPONEN DALAM NEGERI, Pasal 8:\n(1) Penyelenggara telekomunikasi wajib menilai sendiri (self assessment) pencapaian TKDN belanja modal (capex) sebagaimana dimaksud dalam Pasal 3 ayat (2) setiap tahun;\n(2) Penilaian sendiri (self assessment) pencapaian TKDN belanja modal (capex) sebagaimana dimaksud pada ayat (1) dilaksanakan berdasarkan data yang dapat dipertanggung jawabkan meliputi :\n\ta. data yang dimiliki sendiri;\n\tb. data yang dimiliki industri barang/jasa (vendor); dan\n\tc. daftar inventarisasi barang/jasa Komponen dalam negeri yang diterbitkan oleh instansi yang berwenang.\n(3) Komponen yang diajukan dalam penilaian sendiri (self assessment) yang tidak berdasarkan data yang dapat dipertanggungjawabkan, dinyatakan sebagai Komponen luar negeri.",
            "Peraturan Menteri Komunikasi dan Informatika Nomor 41/PER/M.KOMINFO/10/2009 Tahun 2009 tentang Tata Cara Penilaian Pencapaian Tingkat Komponen Dalam Negeri pada Penyelenggaraan Telekomunikasi, BAB V - PELAPORAN, Pasal 10:\n(1) Penyelenggara telekomunikasi wajib melaporkan pencapaian TKDN hasil penilaian sendiri (self assessment) sebagaimana dimaksud dalam Pasal 3 ayat (2) kepada Direktur Jenderal paling lama 3 (tiga) bulan setelah akhir tahun berjalan.\n(2) Laporan pencapaian TKDN sebagaimana dimaksud pada ayat (1) disampaikan sesuai dengan format dengan mengacu pada penjelasan sebagaimana dimaksud dalam Lampiran I dan Lampiran II Peraturan Menteri ini.\n(3) Laporan pencapaian TKDN sebagaimana dimaksud pada ayat (1) harus disertai dengan data pendukung antara lain :\n\ta. dokumen kontrak;\n\tb. kuitansi (invoice);\n\tc. sertifikat negara asal (certificate of origin) ;\n\td. daftar pemasok (vendor);\n\te. sertifikat TKDN.\n(4) Laporan pencapaian TKDN sebagaimana dimaksud pada ayat (1) harus disertai dengan surat pernyataan bermeterai bahwa laporan yang dibuat adalah benar dan akurat serta ditandatangani oleh Direktur Utama atau pejabat yang diberi kewenangan."
        ]
    },
    {
        "user_input": (
            "Apakah Lembaga Penyiaran Asing boleh mendirikan stasiun penyiaran di Indonesia?"
        ),
        "reference_tool_calls": (
            "hybrid_cypher_retriever"
        ),
        "reference": """
            Tidak, Lembaga Penyiaran Asing dilarang didirikan di Indonesia. Namun, mereka dapat menyelenggarakan kegiatan peliputan yang meliputi kegiatan siaran secara tidak tetap dan/atau kegiatan jurnalistik. Kegiatan jurnalistik ini dapat dilakukan dengan menempatkan koresponden atau membuka kantor penyiaran asing.
            """.strip(),
        "reference_contexts": [
            "Peraturan Menteri Komunikasi dan Informatika Nomor 42/PER/M.KOMINFO/10/2009 Tahun 2009 tentang Tata Cara Memperoleh Izin bagi Lembaga Penyiaran Asing yang Melakukan Kegiatan Peliputan di Indonesia, BAB II - KEGIATAN PELIPUTAN, Pasal 2:\nLembaga Penyiaran Asing dilarang didirikan di Indonesia.",
            "Peraturan Menteri Komunikasi dan Informatika Nomor 42/PER/M.KOMINFO/10/2009 Tahun 2009 tentang Tata Cara Memperoleh Izin bagi Lembaga Penyiaran Asing yang Melakukan Kegiatan Peliputan di Indonesia, BAB II - KEGIATAN PELIPUTAN, Pasal 3:\nLembaga Penyiaran Asing hanya dapat menyelenggarakan kegiatan peliputan di Indonesia, yang meliputi :\na. kegiatan, siaran secara tidak tetap ; dan/atau\nb. kegiatan jurnalistik.",
            "Peraturan Menteri Komunikasi dan Informatika Nomor 42/PER/M.KOMINFO/10/2009 Tahun 2009 tentang Tata Cara Memperoleh Izin bagi Lembaga Penyiaran Asing yang Melakukan Kegiatan Peliputan di Indonesia, BAB II - KEGIATAN PELIPUTAN, Pasal 4:\n(1) Lembaga Penyiaran Asing yang menyelenggarakan kegiatan siaran secara tidak tetap di Indonesia sebagaimana dimaksud dalam Pasal 3 dapat membawa perangkat pengiriman ke dan penerima siaran dari satelit dan/atau media lainnya.\n(2) Lembaga Penyiaran Asing yang menyelenggarakan kegiatan jurnalistik di Indonesia sebagaimana dimaksud dalam Pasal 3 dapat :\n\ta. menempatkan koresponden untuk melakukan kegiatan jurnalistik; atau\n\tb. membuka kantor penyiaran asing untuk mendukung bidang administratif.",
            "Peraturan Menteri Komunikasi dan Informatika Nomor 42/PER/M.KOMINFO/10/2009 Tahun 2009 tentang Tata Cara Memperoleh Izin bagi Lembaga Penyiaran Asing yang Melakukan Kegiatan Peliputan di Indonesia, BAB IV - KANTOR PENYIARAN ASING, Pasal 9:\nDalam membuka kantor penyiaran asing sebagaimana dimaksud dalam Pasal 4 ayat (2) huruf b, Lembaga Penyiaran Asing wajib memenuhi ketentuan sebagai berikut :\na. kantor penyiaran asing tersebut bukan merupakan stasiun penyiaran; dan\nb. kantor penyiaran asing tersebut berlokasi di ibukota negara dan berada pada wilayah yurisdiksi Negara Republik Indonesia."
        ]
    },
    {
        "user_input": (
            "Bagaimana cara mendapatkan rekomendasi untuk menyelenggarakan Diklat Radio Elektronika atau Operator Radio (REOR)?"
        ),
        "reference_tool_calls": (
            "hybrid_cypher_retriever"
        ),
        "reference": """
            Lembaga Diklat harus mengajukan permohonan kepada Direktur Jenderal dengan dilengkapi persyaratan administrasi dan perlengkapan teknis, termasuk akta pendirian, NPWP, instruktur berpengalaman, peralatan dan perangkat pendidikan, tempat yang tetap untuk Diklat, dan fasilitas kepustakaan. Direktur Jenderal akan melakukan evaluasi terhadap persyaratan tersebut di lokasi Lembaga Diklat REOR. Lembaga Diklat juga wajib menerapkan kurikulum dan silabus yang ditetapkan oleh Direktur Jenderal.
            """.strip(),
        "reference_contexts": [
            "Peraturan Menteri Komunikasi dan Informatika Nomor 2/PER/M.KOMINFO/3/2011 Tahun 2011 tentang Sertifikasi Radio Elektronika dan Operator Radio, BAB II - PENYELENGGARAAN SERTIFIKASI, Bagian Kedua - Lembaga Pendidikan dan Pelatihan, Pasal 3:\n(1) Pelaksanaan Diklat REOR sebagaimana dimaksud dalam Pasal 2 ayat (3) dilakukan oleh Lembaga Diklat yang telah memenuhi persyaratan sebagai berikut :\n\ta. Lembaga Diklat Pemerintah atau Lembaga Diklat yang berbadan hukum Indonesia; dan\n\tb. mendapatkan rekomendasi penyelenggaraan Diklat dari Direktur Jenderal.\n(2) Untuk mendapatkan rekomendasi sebagaimana dimaksud pada ayat (1) huruf b, Lembaga Diklat mengajukan permohonan kepada Direktur Jenderal dengan dilengkapi persyaratan administrasi dan perlengkapan teknis termasuk sarana dan prasarana sebagai berikut:\n\ta. akta pendirian yang telah disahkan oleh instansi yang berwenang, yang di dalam Anggaran Dasar pendiriannya mencantumkan bidang penyelenggaraan Diklat REOR;\n\tb. Nomor Pokok Wajib Pajak (NPWP);\n\tc. mempunyai instruktur yang berpengalaman sekurang-kurangnya 6 (enam) orang;\n\td. menyediakan seluruh peralatan dan perangkat pendidikan;\n\te. tempat yang tetap untuk menyelenggarakan Diklat; dan\n\tf. mempunyai fasilitas kepustakaan sebagai sarana Diklat.\n(3) Direktur Jenderal melakukan evaluasi persyaratan administrasi dan perlengkapan teknis termasuk sarana dan prasarana sebagaimana dimaksud pada ayat (2) di lokasi Lembaga Diklat REOR.\n(4) Lembaga Diklat REOR sebagaimana dimaksud pada ayat (1) wajib menerapkan kurikulum dan silabus yang ditetapkan oleh Direktur Jenderal.\n(5) Lembaga Diklat REOR sebagaimana dimaksud pada ayat (1) wajib menyampaikan laporan kegiatan penyelenggaraan Diklat REOR per semester kepada Direktur Jenderal.\n(6) Direktur Jenderal membentuk Tim untuk melakukan evaluasi terhadap penerapan kurikulum dan silabus, dan pelaporan kegiatan penyelenggaraan Diklat REOR.\n(7) Tata cara evaluasi terhadap penerapan kurikulum dan silabus, dan pelaporan kegiatan penyelenggaraan Diklat REOR diatur dengan Peraturan Direktur Jenderal.",
            "Peraturan Menteri Komunikasi dan Informatika Nomor 2/PER/M.KOMINFO/3/2011 Tahun 2011 tentang Sertifikasi Radio Elektronika dan Operator Radio, BAB IV - PEMBINAAN DAN PENGAWASAN, Pasal 12:\nDirektorat Jenderal melaksanakan fungsi pembinaan dan pengawasan terhadap pelaksanaan Diklat REOR yang diselenggarakan oleh Lembaga Diklat REOR.",
            "Peraturan Menteri Komunikasi dan Informatika Nomor 2/PER/M.KOMINFO/3/2011 Tahun 2011 tentang Sertifikasi Radio Elektronika dan Operator Radio, BAB V - SANKSI, Pasal 13:\n(1) Lembaga Diklat REOR yang terbukti tidak menerapkan kurikulum dan silabus sebagaimana dimaksud dalam Pasal 3 ayat (4) atau tidak mengirimkan laporan sebagaimana dimaksud dalam Pasal 3 ayat (5), diberi peringatan tertulis sebanyak 3 (tiga) kali dengan tenggang waktu 30 (tiga puluh) hari kalender untuk setiap peringatan.\n(2) Dalam hal Lembaga Diklat REOR telah diperingatkan 3 (tiga) kali berturut-turut dan tidak ditindaklanjuti, maka Lembaga Diklat dimaksud dilarang menyelenggarakan Diklat REOR.\n(3) Calon pemegang Sertifikat Kewenangan yang tidak mengucapkan sumpah atau janji sebagaimana dimaksud dalam Pasal 10 tidak akan diberikan Sertifikat Kewenangan sebagaimana dimaksud dalam Pasal 2 ayat (2)."
        ]
    },
    {
        "user_input": (
            "Apa isi Pasal 10 Peraturan Menteri Komunikasi dan Informatika (PERMENKOMINFO) Nomor 4 Tahun 2016?"
        ),
        "reference_tool_calls": (
            "text2cypher_retriever"
        ),
        "reference": """
            Isi Pasal 10 Peraturan Menteri Komunikasi dan Informatika (PERMENKOMINFO) Nomor 4 Tahun 2016 adalah:
            (1) Penyelenggara Sistem Elektronik strategis dan Penyelenggara Sistem Elektronik tinggi wajib memiliki Sertifikat Sistem Manajemen Pengamanan Informasi.
            (2) Penyelenggara Sistem Elektronik rendah dapat memiliki Sertifikat Sistem Manajemen Pengamanan Informasi.
            """.strip(),
        "reference_contexts": [
            '{"text": "Peraturan Menteri Komunikasi dan Informatika Nomor 4 Tahun 2016 tentang Sistem Manajemen Pengamanan Informasi, BAB IV - PENYELENGGARAAN, Bagian Kesatu - Penyelenggara Sistem Elektronik, Pasal 10:\n(1) Penyelenggara Sistem Elektronik strategis dan Penyelenggara Sistem Elektronik tinggi wajib memiliki Sertifikat Sistem Manajemen Pengamanan Informasi.\n(2) Penyelenggara Sistem Elektronik rendah dapat memiliki Sertifikat Sistem Manajemen Pengamanan Informasi."}'
        ]
    },
    {
        "user_input": (
            "Apa saja daftar subjek peraturan di dalam database? Pakai text2cypher"
        ),
        "reference_tool_calls": (
            "text2cypher_retriever"
        ),
        "reference": """
            Berikut adalah daftar topik atau subjek peraturan yang ada:

            1.  ADMINISTRASI DAN TATA USAHA NEGARA
            2.  CIPTA KERJA
            3.  HAK ASASI MANUSIA
            4.  HAK ATAS KEKAYAAN INTELEKTUAL
            5.  ILMU PENGETAHUAN DAN TEKNOLOGI
            6.  INFORMASI PUBLIK
            7.  JABATAN / PROFESI / KEAHLIAN / SERTIFIKASI
            8.  JARINGAN DOKUMENTASI DAN INFORMASI HUKUM / JDIH
            9.  OTONOMI DAERAH DAN PEMERINTAH DAERAH
            10. PARTAI POLITIK DAN PEMILU
            11. PEMILIHAN UMUM DAERAH, DPRD, PEMILIHAN KEPALA DAERAH
            12. PENDIDIKAN
            13. PENGADAAN BARANG / JASA
            14. PERIZINAN, PELAYANAN PUBLIK
            15. PERLINDUNGAN KONSUMEN
            16. PERLINDUNGAN USAHA, PERUSAHAAN, BADAN USAHA, PERDAGANGAN
            17. PERS, POS, DAN PERIKLANAN
            18. PROGRAM, RENCANA PEMBANGUNAN DAN RENCANA KERJA
            19. SISTEM PENGENDALIAN INTERN
            20. STANDAR / PEDOMAN
            21. STRUKTUR ORGANISASI
            22. TELEKOMUNIKASI, INFORMATIKA, SIBER, DAN INTERNET
            """.strip(),
        "reference_contexts": [
            '{"subject": "TELEKOMUNIKASI, INFORMATIKA, SIBER, DAN INTERNET"}',
            '{"subject": "PERIZINAN, PELAYANAN PUBLIK"}',
            '{"subject": "PERS, POS, DAN PERIKLANAN"}',
            '{"subject": "JABATAN / PROFESI / KEAHLIAN / SERTIFIKASI"}',
            '{"subject": "JARINGAN DOKUMENTASI DAN INFORMASI HUKUM / JDIH"}',
            '{"subject": "PARTAI POLITIK DAN PEMILU"}',
            '{"subject": "PEMILIHAN UMUM DAERAH, DPRD, PEMILIHAN KEPALA DAERAH"}',
            '{"subject": "ILMU PENGETAHUAN DAN TEKNOLOGI"}',
            '{"subject": "HAK ATAS KEKAYAAN INTELEKTUAL"}',
            '{"subject": "OTONOMI DAERAH DAN PEMERINTAH DAERAH"}',
            '{"subject": "STRUKTUR ORGANISASI"}',
            '{"subject": "STANDAR / PEDOMAN"}',
            '{"subject": "PERLINDUNGAN KONSUMEN"}',
            '{"subject": "PENDIDIKAN"}',
            '{"subject": "PROGRAM, RENCANA PEMBANGUNAN DAN RENCANA KERJA"}',
            '{"subject": "ADMINISTRASI DAN TATA USAHA NEGARA"}',
            '{"subject": "SISTEM PENGENDALIAN INTERN"}',
            '{"subject": "PENGADAAN BARANG / JASA"}',
            '{"subject": "CIPTA KERJA"}',
            '{"subject": "PERLINDUNGAN USAHA, PERUSAHAAN, BADAN USAHA, PERDAGANGAN"}',
            '{"subject": "HAK ASASI MANUSIA"}',
            '{"subject": "INFORMASI PUBLIK"}'
        ]
    },
    {
        "user_input": (
            "Apa judul pasal selanjutnya setelah Pasal 39 UU Nomor 11 Tahun 2008?"
        ),
        "reference_tool_calls": (
            "text2cypher_retriever"
        ),
        "reference": """
            Pasal selanjutnya setelah Pasal 39 UU Nomor 11 Tahun 2008 adalah Pasal 40.

            Namun, perlu diperhatikan bahwa terdapat tiga node Pasal 40 yang berbeda, yaitu:
            - Pasal 40 dari UU Nomor 1 Tahun 2024 (status: Berlaku)
            - Pasal 40 dari UU Nomor 11 Tahun 2008 (status: Tidak Berlaku)
            - Pasal 40 dari UU Nomor 19 Tahun 2016 (status: Tidak Berlaku)

            Ini menunjukkan bahwa Pasal 40 dari UU Nomor 11 Tahun 2008 telah diubah oleh UU Nomor 19 Tahun 2016, dan kemudian diubah lagi oleh UU Nomor 1 Tahun 2024. Pasal 40 yang berlaku saat ini adalah Pasal 40 dari UU Nomor 1 Tahun 2024.
            """.strip(),
        "reference_contexts": [
            '{"article": "Pasal 40 UU Nomor 11 Tahun 2008"}',
            '{"article": "Pasal 40 UU Nomor 19 Tahun 2016 tentang Perubahan Atas Undang-Undang Nomor 11 Tahun 2008"}',
            '{"article": "Pasal 40 UU Nomor 1 Tahun 2024 tentang Perubahan Kedua atas Undang-Undang Nomor 11 Tahun 2008"}'
        ]
    },
]

In [ ]:
single_turn_evaluation_dataset = []
multi_turn_evaluation_dataset = []

for data in dataset:
    multi_turn_evaluation_dataset.append({
        "user_input": [HumanMessage(content=data["user_input"])],
        "reference_tool_calls": [
            ToolCall(
                name=data["reference_tool_calls"],
                args={"query": data["user_input"]}
            )
        ]
    })

    single_turn_evaluation_dataset.append({
        "user_input": data["user_input"],
        "reference": data["reference"],
        "reference_contexts": data["reference_contexts"]
    })

single_turn_evaluation_dataset = EvaluationDataset.from_list(single_turn_evaluation_dataset)
multi_turn_evaluation_dataset = EvaluationDataset.from_list(multi_turn_evaluation_dataset)

In [ ]:
graph_rag_eval_result = graph_rag_eval_workflow(
    single_turn_evaluation_dataset,
    multi_turn_evaluation_dataset,
    model=llm,
    neo4j_graph=neo4j_graph,
    total_definition_limit=total_definition_limit,
    tools=[text2cypher_retriever, hybrid_cypher_retriever],
    fallback_tool_calling_cls=FallbackToolCalling,
    verbose=True
)

single_turn_evaluation_result, multi_turn_evaluation_result = graph_rag_eval_result

Running end-to-end graph_rag on evaluation dataset:   0%|          | 0/2 [00:00<?, ?it/s]

Running end-to-end graph_rag on evaluation dataset:  50%|█████     | 1/2 [00:10<00:10, 10.56s/it]

[('hybrid_cypher_retriever', {'run_time': 0.06702065467834473, 'is_context_fetched': True, 'node_ids': [200903041500400, 200703038501801, 200903041500200, 200903041500100, 200903041500300, 200903041501000, 200903041500800, 200903041400700, 200903041400300, 202403003400800, 201803016401800, 199901036400400]})]


Running end-to-end graph_rag on evaluation dataset: 100%|██████████| 2/2 [00:22<00:00, 11.03s/it]

[('text2cypher_retriever', {'is_context_fetched': True, 'context': [{'text': 'Peraturan Menteri Komunikasi dan Informatika Nomor 4 Tahun 2016 tentang Sistem Manajemen Pengamanan Informasi, BAB IV - PENYELENGGARAAN, Bagian Kesatu - Penyelenggara Sistem Elektronik, Pasal 10:\n(1) Penyelenggara Sistem Elektronik strategis dan Penyelenggara Sistem Elektronik tinggi wajib memiliki Sertifikat Sistem Manajemen Pengamanan Informasi.\n(2) Penyelenggara Sistem Elektronik rendah dapat memiliki Sertifikat Sistem Manajemen Pengamanan Informasi.'}], 'run_time': 4.209072113037109})]


In [ ]:
for data in multi_turn_evaluation_result:
    display(data.model_dump())

{'user_input': [{'content': 'Dalam pembangunan infrastruktur telekomunikasi, bagaimana cara perhitungan persentase TKDN untuk belanja modal atau capital expenditure (Capex) bisa dilakukan? Carikan caranya',
   'metadata': None,
   'type': 'human'},
  {'content': 'Untuk mencari tahu cara perhitungan persentase TKDN untuk belanja modal atau Capex dalam pembangunan infrastruktur telekomunikasi, saya akan menggunakan tool `hybrid_cypher_retriever`.',
   'metadata': None,
   'type': 'ai',
   'tool_calls': [{'name': 'hybrid_cypher_retriever',
     'args': {'query': 'bagaimana cara perhitungan persentase TKDN untuk belanja modal atau capital expenditure (Capex) dalam pembangunan infrastruktur telekomunikasi?'}}]},
  {'content': '## **Daftar Pasal Peraturan Perundang-Undangan yang (Mungkin) Relevan untuk Menjawab Kueri:**\n--------------------------------------------------------------------------------\n\nPeraturan Menteri Komunikasi dan Informatika Nomor 41/PER/M.KOMINFO/10/2009 Tahun 2009 te

{'user_input': [{'content': 'Apa isi Pasal 10 Peraturan Menteri Komunikasi dan Informatika (PERMENKOMINFO) Nomor 4 Tahun 2016?',
   'metadata': None,
   'type': 'human'},
  {'content': '',
   'metadata': None,
   'type': 'ai',
   'tool_calls': [{'name': 'text2cypher_retriever',
     'args': {'query': 'isi Pasal 10 Peraturan Menteri Komunikasi dan Informatika (PERMENKOMINFO) Nomor 4 Tahun 2016'}}]},
  {'content': "### **Hasil Pembuatan Kode Cypher:**\n\\`\\`\\`cypher\nMATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)\nWHERE r.type = 'PERMENKOMINFO' AND r.number = 4 AND r.year = 2016 AND a.number = '10'\nRETURN a.text AS text\n\\`\\`\\`\n\n### **Hasil Eksekusi Kode Cypher ke Database:**\n[{'text': 'Peraturan Menteri Komunikasi dan Informatika Nomor 4 Tahun 2016 tentang Sistem Manajemen Pengamanan Informasi, BAB IV - PENYELENGGARAAN, Bagian Kesatu - Penyelenggara Sistem Elektronik, Pasal 10:\\n(1) Penyelenggara Sistem Elektronik strategis dan Penyelenggara Sistem Elektronik tinggi wajib me

In [ ]:
for data in single_turn_evaluation_result:
    display(data.model_dump())

{'user_input': 'Dalam pembangunan infrastruktur telekomunikasi, bagaimana cara perhitungan persentase TKDN untuk belanja modal atau capital expenditure (Capex) bisa dilakukan? Carikan caranya',
 'retrieved_contexts': ['Peraturan Menteri Komunikasi dan Informatika Nomor 41/PER/M.KOMINFO/10/2009 Tahun 2009 tentang Tata Cara Penilaian Pencapaian Tingkat Komponen Dalam Negeri pada Penyelenggaraan Telekomunikasi, BAB III - TINGKAT KOMPONEN DALAM NEGERI, Pasal 4:\n(1) Dalam hal penyelenggara telekomunikasi melakukan penilaian TKDN dengan menggunakan persentase TKDN belanja modal (capex) Komponen material/perangkat telekomunikasi dari penyedia barang/jasa dalam negeri yang menjadi pemasok (vendor) penyelenggara telekomunikasi dalam pembangunan infrastruktur telekomunikasi, harus memperlihatkan bukti-bukti pembelanjaan dalam negeri.\n(2) Persentase TKDN Komponen material/perangkat telekomunikasi dari penyedia barang/jasa dalam negeri sebagaimana dimaksud pada ayat (1) harus sudah disahkan oleh

{'user_input': 'Apa isi Pasal 10 Peraturan Menteri Komunikasi dan Informatika (PERMENKOMINFO) Nomor 4 Tahun 2016?',
 'retrieved_contexts': ["{'text': 'Peraturan Menteri Komunikasi dan Informatika Nomor 4 Tahun 2016 tentang Sistem Manajemen Pengamanan Informasi, BAB IV - PENYELENGGARAAN, Bagian Kesatu - Penyelenggara Sistem Elektronik, Pasal 10:\\n(1) Penyelenggara Sistem Elektronik strategis dan Penyelenggara Sistem Elektronik tinggi wajib memiliki Sertifikat Sistem Manajemen Pengamanan Informasi.\\n(2) Penyelenggara Sistem Elektronik rendah dapat memiliki Sertifikat Sistem Manajemen Pengamanan Informasi.'}"],
 'reference_contexts': ['{"text": "Peraturan Menteri Komunikasi dan Informatika Nomor 4 Tahun 2016 tentang Sistem Manajemen Pengamanan Informasi, BAB IV - PENYELENGGARAAN, Bagian Kesatu - Penyelenggara Sistem Elektronik, Pasal 10:\n(1) Penyelenggara Sistem Elektronik strategis dan Penyelenggara Sistem Elektronik tinggi wajib memiliki Sertifikat Sistem Manajemen Pengamanan Informa

In [ ]:
# from ragas.integrations.langgraph import convert_to_ragas_messages
# from langchain_core.messages import AIMessage, ToolMessage, HumanMessage

# messages = [HumanMessage(content='Apa isi dari pasal 100 UU no 11 tahun 2008?', additional_kwargs={}, response_metadata={}, id='24340d99-986b-40b9-aa34-45c28656d385'),
#  AIMessage(content='', additional_kwargs={'function_call': {'name': 'text2cypher_retriever', 'arguments': '{"query": "isi dari pasal 100 UU no 11 tahun 2008"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, name='agent', id='run-4a3c9eee-f27c-4862-bcca-18a912c5bd69-0', tool_calls=[{'name': 'text2cypher_retriever', 'args': {'query': 'isi dari pasal 100 UU no 11 tahun 2008'}, 'id': '2e757770-b9b8-4810-868a-c73483c0b50e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 433, 'output_tokens': 26, 'total_tokens': 459, 'input_token_details': {'cache_read': 0}}),
#  ToolMessage(content="### **Hasil Pembuatan Kode Cypher:**\n```cypher\nMATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)\nWHERE r.type = 'UU' AND r.number = 11 AND r.year = 2008 AND a.number = '100'\nRETURN a.text AS text\n```\n\n### **Hasil Eksekusi Kode Cypher ke Database:**\n['Tidak dapat menemukan data yang sesuai dengan permintaan query']", name='text2cypher_retriever', id='46f1af24-97ba-4a66-9b8d-991289ab38e9', tool_call_id='2e757770-b9b8-4810-868a-c73483c0b50e', artifact={'is_context_fetched': False, 'cypher_gen_usage_metadata': {'input_tokens': 2855, 'output_tokens': 66, 'total_tokens': 2921, 'input_token_details': {'cache_read': 0}, 'model': 'models/gemini-2.0-flash'}, 'qa_usage_metadata': {}, 'run_time': 0.9165010452270508}),
#  AIMessage(content='', additional_kwargs={}, response_metadata={}, name='agent', id='run-8fd1db57-99e1-4b95-a670-2c7cc6dfbc60-0', tool_calls=[{'name': 'hybrid_cypher_retriever', 'args': {'query': 'isi dari pasal 100 UU no 11 tahun 2008'}, 'id': '94d38894-2ad8-4cf7-9e99-7babe803cd21', 'type': 'tool_call'}]),
#  ToolMessage(content='## **Daftar Pasal Peraturan Perundang-Undangan yang (Mungkin) Relevan untuk Menjawab Kueri:**\n--------------------------------------------------------------------------------\n\nUndang-undang (UU) Nomor 11 Tahun 2008 tentang Informasi dan Transaksi Elektronik, BAB III - INFORMASI DOKUMEN DAN TANDA TANGAN ELEKTRONIK, Pasal 7:\nSetiap Orang yang menyatakan hak, memperkuat hak yang telah ada, atau menolak hak Orang lain berdasarkan adanya Informasi Elektronik dan/atau Dokumen Elektronik harus memastikan bahwa Informasi Elektronik dan/atau Dokumen Elektronik yang ada padanya berasal dari Sistem Elektronik yang memenuhi syarat berdasarkan Peraturan Perundangundangan.\n\nUndang-undang (UU) Nomor 36 Tahun 1999 tentang Telekomunikasi, BAB VII - KETENTUAN PIDANA, Pasal 59:\nPerbuatan sebagaimana dimaksud dalam Pasal 47, Pasal 48, Pasal 49, Pasal 50, Pasal 51,Pasal 52, Pasal 53, Pasal 54, Pasal 55, Pasal 56, dan Pasal 57 adalah kejahatan.\n\nUndang-undang (UU) Nomor 11 Tahun 2008 tentang Informasi dan Transaksi Elektronik, BAB V - TRANSAKSI ELEKTRONIK, Pasal 19:\nPara pihak yang melakukan Transaksi Elektronik harus menggunakan Sistem Elektronik yang disepakati.\n\nPeraturan Menteri Komunikasi dan Informatika Nomor 5 Tahun 2020 tentang Penyelenggara Sistem Elektronik Lingkup Privat, BAB II - PENDAFTARAN PENYELENGGARA SISTEM ELEKTRONIK LINGKUP PRIVAT, Bagian Kesatu - Pendaftaran Penyelenggara Sistem Elektronik Lingkup Privat, Pasal 5:\nPerubahan terhadap informasi pendaftaran sebagaimana dimaksud dalam Pasal 3 ayat (3), Pasal 3 ayat (5), dan Pasal 4 ayat (2) wajib dilaporkan kepada Menteri.\n\nPeraturan Menteri Komunikasi dan Informatika Nomor 10 Tahun 2016 tentang Layanan Nomor Tunggal Panggilan Darurat, BAB VI - SANKSI, Pasal 24:\nPelanggaran terhadap ketentuan sebagaimana dimaksud dalam Pasal 8 ayat (1), Pasal 9, Pasal 11, Pasal 12, Pasal 14 ayat (3) dan Pasal 20 dikenai sanksi sesuai dengan ketentuan peraturan perundang-undangan.\n\nUndang-undang (UU) Nomor 36 Tahun 1999 tentang Telekomunikasi, BAB VII - KETENTUAN PIDANA, Pasal 54:\nBarang siapa yang melanggar ketentuan sebagaimana dimaksud dalam Pasal 35 ayat (2) atau Pasal 36 ayat (2), dipidana dengan pidana penjara paling lama 2 (dua) tahun dan atau denda paling banyak Rp200.000.000,00 (dua ratus juta rupiah).\n\nUndang-undang (UU) Nomor 36 Tahun 1999 tentang Telekomunikasi, BAB VII - KETENTUAN PIDANA, Pasal 47:\nBarang siapa yang melanggar ketentuan sebagaimana dimaksud dalam Pasal 11 ayat (1), dipidana dengan pidana penjara paling lama 6 (enam) tahun dan atau denda paling banyak Rp600.000.000,00 (enam ratus juta rupiah).\n\nUndang-undang (UU) Nomor 36 Tahun 1999 tentang Telekomunikasi, BAB VII - KETENTUAN PIDANA, Pasal 48:\nPenyelenggara jaringan telekomunikasi yang melanggar ketentuan sebagaimana dimaksud dalam Pasal 19 dipidana dengan pidana penjara paling lama 1 (satu) tahun dan atau denda paling banyak Rp100.000.000,00 (seratus juta rupiah).\n\nPeraturan Menteri Komunikasi dan Informatika Nomor 10 Tahun 2016 tentang Layanan Nomor Tunggal Panggilan Darurat, BAB III - PENYELENGGARA JARINGAN TELEKOMUNIKASI DALAM LAYANAN NOMOR TUNGGAL PANGGILAN DARURAT, Pasal 11:\nPenyelenggara Jaringan Telekomunikasi dalam menghubungkan jaringan dengan Pusat Panggilan Darurat wajib memenuhi konfigurasi jaringan sebagaimana dimaksud dalam Pasal 10 ayat (1).\n\nUndang-undang (UU) Nomor 36 Tahun 1999 tentang Telekomunikasi, BAB VII - KETENTUAN PIDANA, Pasal 49:\nPenyelenggara telekomunikasi yang melanggar ketentuan sebagaimana dimaksud dalam Pasal 20, dipidana dengan pidana penjara paling lama 2 (dua) tahun dan atau denda paling banyak Rp200.000.000,00 (dua ratus juta rupiah).\n\nUndang-undang (UU) Nomor 36 Tahun 1999 tentang Telekomunikasi, BAB VII - KETENTUAN PIDANA, Pasal 57:\nPenyelenggara jasa telekomunikasi yang melanggar ketentuan sebagaimana dimaksud dalam Pasal 42 ayat (1), dipidana dengan pidana penjara paling lama 2 (dua) tahun dan atau denda paling banyak Rp200.000.000,00 (dua ratus juta rupiah).\n\nUndang-undang (UU) Nomor 36 Tahun 1999 tentang Telekomunikasi, BAB VII - KETENTUAN PIDANA, Pasal 52:\nBarang siapa memperdagangkan, membuat, merakit, memasukkan atau menggunakan perangkat telekomunikasi di wilayah Negara Republik Indonesia yang tidak sesuai dengan persyaratan teknis sebagaimana dimaksud dalam Pasal 32 ayat (1), dipidana dengan pidana penjara paling lama 1 (satu) tahun dan atau denda paling banyak Rp100.000.000,00 (seratus juta rupiah).\n\nUndang-undang (UU) Nomor 36 Tahun 1999 tentang Telekomunikasi, BAB VII - KETENTUAN PIDANA, Pasal 55:\nBarang siapa yang melanggar ketentuan sebagaimana dimaksud dalam Pasal 38, dipidana dengan pidana penjara paling lama 6 (enam) tahun dan atau denda paling banyak Rp600.000.000,00 (enam ratus juta rupiah).\n\nUndang-undang (UU) Nomor 36 Tahun 1999 tentang Telekomunikasi, BAB VII - KETENTUAN PIDANA, Pasal 50:\nBarang siapa yang melanggar ketentuan sebagaimana dimaksud dalam Pasal 22, dipidana dengan pidana penjara paling lama 6 (enam) tahun dan atau denda paling banyak Rp600.000.000,00 (enam ratus juta rupiah).\n\nUndang-undang (UU) Nomor 36 Tahun 1999 tentang Telekomunikasi, BAB VII - KETENTUAN PIDANA, Pasal 51:\nPenyelenggara telekomunikasi khusus yang melanggar ketentuan sebagaimana dimaksud dalam Pasal 29 ayat (1) atau Pasal 29 ayat (2), dipidana dengan pidana penjara paling lama 4 (empat) tahun dan atau denda paling banyak Rp400.000.000,00 (empat ratus juta rupiah).\n\nUndang-undang (UU) Nomor 36 Tahun 1999 tentang Telekomunikasi, BAB VII - KETENTUAN PIDANA, Pasal 53:\n(1) Barang siapa yang melanggar ketentuan sebagaimana dimaksud dalam Pasal 33 ayat (1) atau Pasal 33 ayat (2), dipidana dengan pidana penjara paling lama 4 (empat) tahun dan atau denda paling banyak Rp400.000.000,00 (empat ratus juta rupiah).\n(2) Apabila tindak pidana sebagaimana dimaksud pada ayat (1) mengakibatkan matinya seseorang, dipidana dengan pidana penjara paling lama 15 (lima belas) tahun.\n\nUndang-undang (UU) Nomor 36 Tahun 1999 tentang Telekomunikasi, BAB VII - KETENTUAN PIDANA, Pasal 56:\nBarang siapa yang melanggar ketentuan sebagaimana dimaksud dalam Pasal 40, dipidana dengan pidana penjara paling lama 15 (lima belas) tahun.\n\nPeraturan Menteri Komunikasi dan Informatika Nomor 5 Tahun 2020 tentang Penyelenggara Sistem Elektronik Lingkup Privat, BAB II - PENDAFTARAN PENYELENGGARA SISTEM ELEKTRONIK LINGKUP PRIVAT, Bagian Ketiga - Penjatuhan Sanksi Administratif dan Normalisasi, Pasal 7:\n(1) Menteri mengenakan sanksi administratif kepada PSE Lingkup Privat yang:\n\ta. tidak melakukan pendaftaran sebagaimana dimaksud dalam Pasal 2 dan Pasal 4;\n\tb. telah mempunyai tanda daftar tetapi tidak melaporkan perubahan terhadap informasi pendaftaran sebagaimana dimaksud dalam Pasal 5;\n\tc. tidak memberikan informasi pendaftaran sebagaimana dimaksud dalam Pasal 3 ayat (3), Pasal 3 ayat (4), dan Pasal 4 ayat (2) dengan benar.\n(2) Dalam hal PSE Lingkup Privat tidak melakukan pendaftaran sebagaimana dimaksud pada ayat (1) huruf a, Menteri memberikan sanksi administratif berupa Pemutusan Akses terhadap Sistem Elektronik (access blocking).\n(3) Dalam hal PSE Lingkup Privat telah mempunyai tanda daftar tetapi tidak melaporkan perubahan terhadap informasi pendaftaran sebagaimana dimaksud pada ayat (1) huruf b atau tidak memberikan informasi pendaftaran dengan benar sebagaimana dimaksud pada ayat (1) huruf c, Menteri memberikan sanksi administratif berupa:\n\ta. teguran tertulis yang disampaikan melalui surat elektronik (electronic mail) dan/atau media elektronik lainnya;\n\tb. penghentian sementara terhadap PSE Lingkup Privat dalam hal tidak mengindahkan teguran tertulis sebagaimana dimaksud pada ayat (3) huruf a;\n\tc. Pemutusan Akses terhadap Sistem Elektronik (access blocking) dan pencabutan Tanda Daftar Penyelenggara Sistem Elektronik dalam hal PSE Lingkup Privat tidak memberikan konfirmasi dalam jangka waktu 7 (tujuh) hari setelah penghentian sementara sebagaimana dimaksud pada ayat (3) huruf b.\n(4) Dalam hal PSE Lingkup Privat telah memenuhi ketentuan pendaftaran sebagaimana dimaksud dalam Pasal 2 sampai dengan Pasal 5, Menteri melakukan Normalisasi terhadap Sistem Elektronik yang diputus aksesnya (access blocking) sebagaimana dimaksud pada ayat (2).\n(5) Dalam hal PSE Lingkup Privat telah melakukan pembaruan informasi pendaftaran dengan benar, Menteri melakukan Normalisasi terhadap Sistem Elektronik yang dihentikan sementara sebagaimana dimaksud pada ayat (3) huruf b.\n(6) Dalam hal PSE Lingkup Privat telah melakukan pendaftaran ulang dengan memberikan informasi pendaftaran dengan benar, Menteri melakukan Normalisasi terhadap Sistem Elektronik yang diputus akses Sistem Elektroniknya dan dicabut tanda daftar Penyelenggara Sistem Elektroniknya sebagaimana dimaksud pada ayat (3) huruf c.\n\nPeraturan Menteri Komunikasi dan Informatika Nomor 10 Tahun 2016 tentang Layanan Nomor Tunggal Panggilan Darurat, BAB III - PENYELENGGARA JARINGAN TELEKOMUNIKASI DALAM LAYANAN NOMOR TUNGGAL PANGGILAN DARURAT, Pasal 9:\nPenyelenggara Jaringan Telekomunikasi sebagaimana dimaksud dalam Pasal 8 ayat (1) wajib:\na. menyediakan Jaringan, Sarana dan Prasarana Telekomunikasi yang terhubung dengan Pusat Panggilan Darurat;\nb. menyambungkan panggilan keadaan darurat yang diterima dari masyarakat ke Pusat Panggilan Darurat di Provinsi atau Kabupaten/Kota sesuai lokasi pemanggil; dan\nc. menginformasikan lokasi dan nomor telepon pemanggil ke pusat data nasional panggilan darurat.\n\nPeraturan Menteri Komunikasi dan Informatika Nomor 5 Tahun 2020 tentang Penyelenggara Sistem Elektronik Lingkup Privat, BAB II - PENDAFTARAN PENYELENGGARA SISTEM ELEKTRONIK LINGKUP PRIVAT, Bagian Kedua - Penerbitan Tanda Daftar, Pasal 6:\n(1) Tanda daftar PSE Lingkup Privat diterbitkan oleh Menteri setelah persyaratan pendaftaran sebagaimana dimaksud dalam Pasal 2 sampai dengan Pasal 5 dinyatakan lengkap sesuai dengan Peraturan Menteri ini dan ditempatkan dalam daftar PSE Lingkup Privat.\n(2) Daftar PSE Lingkup Privat sebagaimana dimaksud pada ayat (1) dimuat di laman website yang dikelola oleh Kementerian.\n\nPeraturan Menteri Komunikasi dan Informatika Nomor 5 Tahun 2020 tentang Penyelenggara Sistem Elektronik Lingkup Privat, BAB II - PENDAFTARAN PENYELENGGARA SISTEM ELEKTRONIK LINGKUP PRIVAT, Bagian Kesatu - Pendaftaran Penyelenggara Sistem Elektronik Lingkup Privat, Pasal 4:\n(1) Kewajiban PSE Lingkup Privat melakukan pendaftaran sebagaimana dimaksud dalam Pasal 2 ayat (1) juga berlaku untuk PSE Lingkup Privat yang didirikan menurut hukum negara lain atau yang berdomisili tetap di negara lain tetapi:\n\ta. memberikan layanan di dalam wilayah Indonesia;\n\tb. melakukan usaha di Indonesia; dan/atau\n\tc. Sistem Elektroniknya dipergunakan dan/atau ditawarkan di wilayah Indonesia.\n(2) Pendaftaran PSE Lingkup Privat sebagaimana dimaksud pada ayat (1) dilakukan dengan mengisi formulir pendaftaran yang memuat informasi sebagaimana dimaksud dalam Pasal 3 ayat (3) dan Pasal 3 ayat (4) serta informasi yang benar yang meliputi:\n\ta. identitas PSE Lingkup Privat;\n\tb. identitas pimpinan perusahaan dan/atau identitas penanggung jawab;\n\tc. keterangan domisili dan/atau akta pendirian perusahaan (certificate of incorporation);\n\td. jumlah pelanggan (user) dari Indonesia; dan\n\te. nilai transaksi yang berasal dari Indonesia.\n(3) Informasi sebagaimana dimaksud pada ayat (2) huruf c disampaikan dengan melampirkan dokumen pendukung yang diterjemahkan ke dalam bahasa Indonesia oleh penerjemah tersumpah.\n\nPeraturan Menteri Komunikasi dan Informatika Nomor 10 Tahun 2016 tentang Layanan Nomor Tunggal Panggilan Darurat, BAB III - PENYELENGGARA JARINGAN TELEKOMUNIKASI DALAM LAYANAN NOMOR TUNGGAL PANGGILAN DARURAT, Pasal 14:\n(1) Pemerintah Daerah Provinsi atau Kabupaten/Kota dapat mengusulkan penambahan kapasitas jaringan telekomunikasi sebagaimana dimaksud dalam Pasal 10 ayat (2) untuk meningkatkan kualitas layanan Nomor Tunggal Panggilan Darurat.\n(2) Usulan penambahan kapasitas jaringan telekomunikasi sebagaimana dimaksud pada ayat (1) harus terlebih dahulu disetujui oleh Penyelenggara Jaringan Telekomunikasi.\n(3) Penyelenggara Jaringan Telekomunikasi wajib melaporkan secara tertulis kepada Direktur Jenderal setiap usulan penambahan kapasitas jaringan telekomunikasi di Daerah Provinsi atau Kabupaten/Kota paling lambat 1 (satu) bulan sejak usulan penambahan kapasitas diterima Penyelenggara Jaringan Telekomunikasi.\n\nPeraturan Menteri Komunikasi dan Informatika Nomor 10 Tahun 2016 tentang Layanan Nomor Tunggal Panggilan Darurat, BAB IV - SISTEM PUSAT PANGGILAN DARURAT, Pasal 20:\nSetiap orang yang melakukan panggilan ke Pusat Panggilan Darurat dilarang memberikan informasi yang tidak benar.\n\nPeraturan Menteri Komunikasi dan Informatika Nomor 10 Tahun 2016 tentang Layanan Nomor Tunggal Panggilan Darurat, BAB III - PENYELENGGARA JARINGAN TELEKOMUNIKASI DALAM LAYANAN NOMOR TUNGGAL PANGGILAN DARURAT, Pasal 8:\n(1) Penyelenggara Jaringan Telekomunikasi wajib menyampaikan panggilan ke Pusat Panggilan Darurat di Provinsi atau Kabupaten/Kota sesuai lokasi pemanggil.\n(2) Penyelenggara Jaringan Telekomunikasi sebagaimana dimaksud pada ayat (1) meliputi:\n\ta. penyelenggara jaringan tetap lokal berbasis circuitswitched;\n\tb. penyelenggara jaringan bergerak seluler; dan\n\tc. penyelenggara jaringan bergerak satelit.\n\nPeraturan Menteri Komunikasi dan Informatika Nomor 10 Tahun 2016 tentang Layanan Nomor Tunggal Panggilan Darurat, BAB III - PENYELENGGARA JARINGAN TELEKOMUNIKASI DALAM LAYANAN NOMOR TUNGGAL PANGGILAN DARURAT, Pasal 12:\nPersentase jumlah panggilan layanan Nomor Tunggal Darurat yang tidak mengalami Dropped Call pada jaringan Penyelenggara Jaringan Telekomunikasi wajib paling sedikit sesuai dengan ketentuan peraturan perundang-undangan.\n\nPeraturan Menteri Komunikasi dan Informatika Nomor 5 Tahun 2020 tentang Penyelenggara Sistem Elektronik Lingkup Privat, BAB II - PENDAFTARAN PENYELENGGARA SISTEM ELEKTRONIK LINGKUP PRIVAT, Bagian Kesatu - Pendaftaran Penyelenggara Sistem Elektronik Lingkup Privat, Pasal 3:\n(1) Pendaftaran PSE Lingkup Privat sebagaimana dimaksud dalam Pasal 2 ayat (1) diajukan kepada Menteri.\n(2) Pengajuan permohonan pendaftaran PSE Lingkup Privat sebagaimana dimaksud pada ayat (1) dilakukan melalui OSS, kecuali yang ditentukan lain oleh ketentuan peraturan perundang-undangan.\n(3) Pengajuan permohonan pendaftaran sebagaimana dimaksud pada ayat (2) dilakukan dengan mengisi formulir pendaftaran yang memuat informasi yang benar mengenai:\n\ta. gambaran umum pengoperasian Sistem Elektronik;\n\tb. kewajiban untuk memastikan keamanan informasi sesuai dengan ketentuan peraturan perundang- undangan;\n\tc. kewajiban melakukan pelindungan Data Pribadi sesuai dengan ketentuan peraturan perundang- undangan; dan\n\td. kewajiban untuk melakukan uji kelaikan Sistem Elektronik sesuai dengan ketentuan peraturan perundang-undangan.\n(4) Informasi mengenai gambaran umum pengoperasian Sistem Elektronik sebagaimana dimaksud pada ayat (3) huruf a, terdiri atas:\n\ta. nama Sistem Elektronik;\n\tb. sektor Sistem Elektronik;\n\tc. uniform resource locator (URL) website;\n\td. sistem nama domain (domain name system) dan/atau alamat Internet Protocol (IP) server;\n\te. deskripsi model bisnis;\n\tf. deskripsi singkat fungsi Sistem Elektronik dan proses bisnis Sistem Elektronik;\n\tg. keterangan Data Pribadi yang diproses;\n\th. keterangan lokasi pengelolaan, pemrosesan, dan/atau penyimpanan Sistem Elektronik dan Data Elektronik; dan\n\ti. keterangan yang menyatakan bahwa PSE Lingkup Privat menjamin dan melaksanakan kewajiban pemberian akses terhadap Sistem Elektronik dan Data Elektronik dalam rangka memastikan efektivitas pengawasan dan penegakan hukum sesuai dengan ketentuan peraturan perundang- undangan.\n(5) Pendaftaran PSE Lingkup Privat yang dikecualikan untuk melakukan pendaftaran melalui OSS sebagaimana dimaksud pada ayat (2) dilakukan dengan menyampaikan informasi sebagaimana dimaksud pada ayat (3) dan ayat (4) serta menyampaikan informasi yang benar mengenai:\n\ta. nama badan hukum, alamat badan hukum, bentuk badan hukum, akta perusahaan dan akta perubahan terakhir;\n\tb. nomor pokok wajib pajak;\n\tc. nama, nomor induk kependudukan, dan nomor telepon; dan\n\td. keterangan yang menyatakan bahwa PSE Lingkup Privat telah memiliki legalitas dalam menyelenggarakan kegiatan berusaha dari Kementerian atau Lembaga yang memiliki kewenangan sesuai dengan ketentuan peraturan perundang-undangan yang dibuktikan dengan dokumen terkait.\n\n\n## **Daftar Definisi Konsep Menurut Peraturan Perundang-Undangan yang (Mungkin) Relevan untuk Menjawab Kueri:**\n--------------------------------------------------------------------------------\n\n- Definisi "Tahun Buku" Menurut Pasal 1 Peraturan Menteri Komunikasi dan Informatika Nomor 1 Tahun 2021 tentang Perubahan Kedua atas Peraturan Menteri Komunikasi dan Informatika Nomor 13 Tahun 2019 tentang Penyelenggaraan Jasa Telekomunikasi:\nTahun Buku adalah jangka waktu 1 (satu) tahun yang dimulai dari bulan Januari sampai dengan bulan Desember.\n\n- Definisi "Penerima" Menurut Pasal 1 Undang-undang (UU) Nomor 11 Tahun 2008 tentang Informasi dan Transaksi Elektronik:\nPenerima adalah subjek hukum yang menerima Informasi Elektronik dan/atau Dokumen Elektronik dari Pengirim.\n\n- Definisi "Tahun Buku" Menurut Pasal 1 Peraturan Menteri Komunikasi dan Informatika Nomor 13 Tahun 2019 tentang Penyelenggaraan Jasa Telekomunikasi:\nTahun Buku adalah jangka waktu 1 (satu) tahun yang dimulai dari bulan Januari sampai dengan bulan Desember.\n\n- Definisi "Aplikasi Umum" Menurut Pasal 1 Peraturan Menteri Komunikasi dan Informatika Nomor 16 Tahun 2022 tentang Kebijakan Umum Penyelenggaraan Audit Teknologi Informasi dan Komunikasi:\nAplikasi Umum adalah Aplikasi SPBE yang sama, standar, dan digunakan secara bagi pakai oleh Instansi Pusat dan/atau Pemerintah Daerah.\n\n- Definisi "Lembaga Pelatihan Pemerintah" Menurut Pasal 1 Peraturan Menteri Komunikasi dan Informatika Nomor 1 Tahun 2018 tentang Akreditasi Lembaga Pelatihan Teknis Bidang Teknologi Informasi dan Komunikasi:\nLembaga Pelatihan Pemerintah yang selanjutnya disebut Lembaga Pelatihan adalah satuan unit organisasi penyelenggara fungsi Pelatihan bagi Aparatur Sipil Negara, baik yang mandiri maupun tidak mandiri pada Instansi Pemerintah.', name='hybrid_cypher_retriever', id='e6624bad-f783-4856-af73-d5e932a77682', tool_call_id='94d38894-2ad8-4cf7-9e99-7babe803cd21', artifact={'run_time': 0.06505775451660156, 'is_context_fetched': True, 'node_ids': [200801011500700, 199901036505900, 200801011501900, 202003005500500, 201603010502400, 199901036505400, 199901036504700, 199901036504800, 201603010501100, 199901036504900, 199901036505700, 199901036505200, 199901036505500, 199901036505000, 199901036505100, 199901036505300, 199901036505600, 202003005500700, 201603010500900, 202003005500600, 202003005500400, 201603010501400, 201603010502000, 201603010500800, 201603010501200, 202003005500300, 202103001404200, 200801011401900, 201903013403900, 202203016400900, 201803001400100]}),
#  AIMessage(content='Maaf, saya tidak dapat menemukan pasal 100 dari UU no 11 tahun 2008. Namun, saya menemukan beberapa pasal lain dari UU no 11 tahun 2008 yang mungkin relevan untuk pertanyaan Anda:\n\n*   Undang-undang (UU) Nomor 11 Tahun 2008 tentang Informasi dan Transaksi Elektronik, BAB III - INFORMASI DOKUMEN DAN TANDA TANGAN ELEKTRONIK, Pasal 7:\n    Setiap Orang yang menyatakan hak, memperkuat hak yang telah ada, atau menolak hak Orang lain berdasarkan adanya Informasi Elektronik dan/atau Dokumen Elektronik harus memastikan bahwa Informasi Elektronik dan/atau Dokumen Elektronik yang ada padanya berasal dari Sistem Elektronik yang memenuhi syarat berdasarkan Peraturan Perundangundangan.\n*   Undang-undang (UU) Nomor 11 Tahun 2008 tentang Informasi dan Transaksi Elektronik, BAB V - TRANSAKSI ELEKTRONIK, Pasal 19:\n    Para pihak yang melakukan Transaksi Elektronik harus menggunakan Sistem Elektronik yang disepakati.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, name='agent', id='run-77a08acb-8e59-4fab-8696-95cd3bbd24da-0', usage_metadata={'input_tokens': 4998, 'output_tokens': 220, 'total_tokens': 5218, 'input_token_details': {'cache_read': 0}})]

# convert_to_ragas_messages(messages=messages)

[HumanMessage(content='Apa isi dari pasal 100 UU no 11 tahun 2008?', metadata=None, type='human'),
 AIMessage(content='', metadata=None, type='ai', tool_calls=[]),
 ToolMessage(content="### **Hasil Pembuatan Kode Cypher:**\n```cypher\nMATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)\nWHERE r.type = 'UU' AND r.number = 11 AND r.year = 2008 AND a.number = '100'\nRETURN a.text AS text\n```\n\n### **Hasil Eksekusi Kode Cypher ke Database:**\n['Tidak dapat menemukan data yang sesuai dengan permintaan query']", metadata=None, type='tool'),
 AIMessage(content='', metadata=None, type='ai', tool_calls=None),
 ToolMessage(content='## **Daftar Pasal Peraturan Perundang-Undangan yang (Mungkin) Relevan untuk Menjawab Kueri:**\n--------------------------------------------------------------------------------\n\nUndang-undang (UU) Nomor 11 Tahun 2008 tentang Informasi dan Transaksi Elektronik, BAB III - INFORMASI DOKUMEN DAN TANDA TANGAN ELEKTRONIK, Pasal 7:\nSetiap Orang yang menyatakan hak, memperku

In [ ]:
from ragas import evaluate
from ragas.metrics import ToolCallAccuracy

tool_accuracy = ToolCallAccuracy()

results = evaluate(
    dataset=multi_turn_evaluation_result,
    metrics=[tool_accuracy]
)

results

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

{'tool_call_accuracy': 0.0000}

In [ ]:
from ragas.messages import AIMessage

expected = []
predicted = []
success = 0
failed = 0

for data, multi_turn in list(
    zip(
        [dataset[i] for i in [0, 3]],
        multi_turn_evaluation_result
    )
):
    tool_call_names = []
    for message in multi_turn.user_input:
        if isinstance(message, AIMessage) and message.tool_calls:
            for tool_call in message.tool_calls:
                tool_call_names.append(tool_call.name)

    expected.append([data["reference_tool_calls"]])
    predicted.append(tool_call_names)

    if [data["reference_tool_calls"]] == tool_call_names:
        success += 1
    else:
        failed += 1

print(success)
print(failed)

# ======================== HARUSNYA 100% KAYA GINI ===============================

2
0


In [ ]:
from sklearn.metrics import classification_report

print(expected)
print(predicted, "\n")

print(classification_report(expected, predicted))

[['hybrid_cypher_retriever'], ['text2cypher_retriever']]
[['hybrid_cypher_retriever'], ['text2cypher_retriever']] 

                         precision    recall  f1-score   support

hybrid_cypher_retriever       1.00      1.00      1.00         1
  text2cypher_retriever       1.00      1.00      1.00         1

               accuracy                           1.00         2
              macro avg       1.00      1.00      1.00         2
           weighted avg       1.00      1.00      1.00         2



In [ ]:
# FINAL RESULT
from ragas.metrics import BleuScore
from ragas.metrics import RougeScore
from ragas.metrics import AnswerAccuracy
from ragas.metrics._factual_correctness import FactualCorrectness

factual_correctnes = FactualCorrectness()
answer_accuracy = AnswerAccuracy()
bleu_score = BleuScore(language="indonesia")
rouge1_precision = RougeScore(name="rouge1_precision", rouge_type="rouge1", mode="precision")
rouge1_recall =    RougeScore(name="rouge1_recall",    rouge_type="rouge1", mode="recall")
rouge1_fmeasure =  RougeScore(name="rouge1_fmeasure",  rouge_type="rouge1", mode="fmeasure")
rougeL_precision = RougeScore(name="rougeL_precision", rouge_type="rougeL", mode="precision")
rougeL_recall =    RougeScore(name="rougeL_recall",    rouge_type="rougeL", mode="recall")
rougeL_fmeasure =  RougeScore(name="rougeL_fmeasure",  rouge_type="rougeL", mode="fmeasure")

# HYBRID CYPHER RETRIEVER
from ragas.metrics import NonLLMContextRecall
from ragas.metrics import NonLLMContextPrecisionWithReference

non_llm_context_precision = NonLLMContextPrecisionWithReference(threshold=0.7)
non_llm_context_recall = NonLLMContextRecall(threshold=0.7)


# TEXT2CYPHER RETRIEVER
from ragas.metrics import AspectCritic
from ragas.metrics import SimpleCriteriaScore
from ragas.metrics.base import MetricType
from typing import (
    Dict,
    Set
)

required_columns: Dict[MetricType, Set[str]] = {
    MetricType.SINGLE_TURN: {
        "user_input",
        "reference",
        "retrieved_contexts",
    }
}

aspect_critic = AspectCritic(
    name="kesesuaian_tipe_1",
    definition="Apakah data yang tersedia di `retrieved_contexts` bisa digunakan untuk menjawab `user_input` untuk menghasilkan jawaban `reference`",
    required_columns=required_columns,
    # strictness=2  # default = 1
)

simple_criteria = SimpleCriteriaScore(
    name="kesesuaian_tipe_2",
    definition="Beri nilai 0-100, seberapa baik `retrieved_contexts` bisa digunakan untuk menjawab `user_input` untuk menghasilkan jawaban `reference`",
    required_columns=required_columns,
    # strictness=2  # default = 1
)

from ragas import evaluate

evaluation_result_gemini = evaluate(
    dataset=single_turn_evaluation_result,
    metrics=[
        # factual_correctnes,
        answer_accuracy,
        bleu_score,
        rouge1_precision,
        rouge1_recall,
        rouge1_fmeasure,
        rougeL_precision,
        rougeL_recall,
        rougeL_fmeasure,

        non_llm_context_precision,
        non_llm_context_recall,

        aspect_critic,
        simple_criteria
    ],
    llm=gemini_llm_evaluator
)

evaluation_result_gemini

Evaluating:   0%|          | 0/24 [00:00<?, ?it/s]

{'nv_accuracy': 1.0000, 'bleu_score': 0.5330, 'rouge1_precision(mode=precision)': 0.7066, 'rouge1_recall(mode=recall)': 0.7568, 'rouge1_fmeasure(mode=fmeasure)': 0.7262, 'rougeL_precision(mode=precision)': 0.6708, 'rougeL_recall(mode=recall)': 0.7092, 'rougeL_fmeasure(mode=fmeasure)': 0.6854, 'non_llm_context_precision_with_reference': 0.6603, 'non_llm_context_recall': 0.8750, 'kesesuaian_tipe_1': 1.0000, 'kesesuaian_tipe_2': 100.0000}

In [ ]:
evaluation_result_gemini.to_pandas()

,user_input,retrieved_contexts,reference_contexts,response,reference,nv_accuracy,bleu_score,rouge1_precision(mode=precision),rouge1_recall(mode=recall),rouge1_fmeasure(mode=fmeasure),rougeL_precision(mode=precision),rougeL_recall(mode=recall),rougeL_fmeasure(mode=fmeasure),non_llm_context_precision_with_reference,non_llm_context_recall,kesesuaian_tipe_1,kesesuaian_tipe_2
0,Dalam pembangunan infrastruktur telekomunikasi...,[Peraturan Menteri Komunikasi dan Informatika ...,[Peraturan Menteri Komunikasi dan Informatika ...,"Berdasarkan peraturan yang ada, perhitungan pe...",Persentase TKDN untuk belanja modal (capex) di...,1.0,0.227502,0.43750,0.583333,0.500000,0.366071,0.488095,0.418367,0.320635,0.75,1,100
1,Apa isi Pasal 10 Peraturan Menteri Komunikasi ...,[{'text': 'Peraturan Menteri Komunikasi dan In...,"[{""text"": ""Peraturan Menteri Komunikasi dan In...",Pasal 10 Peraturan Menteri Komunikasi dan Info...,Isi Pasal 10 Peraturan Menteri Komunikasi dan ...,1.0,0.838515,0.97561,0.930233,0.952381,0.975610,0.930233,0.952381,1.000000,1.00,1,100
